# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

import plotly.graph_objects as go
import plotly.express as px

import ta

scaler = MinMaxScaler(feature_range=(0,1))

import pytz
from persiantools.jdatetime import JalaliDate

# Utils Function to process data and convert to jalali

In [2]:
def convert_date(string):
    string = string.split('-')
    return JalaliDate.to_jalali(int(string[0]),int(string[1]),int(string[2])).isoformat()
    

def date_preprocess(date):
    date = str(date)
    return date[0:4]+'-'+date[4:6]+'-'+date[6:]

# Read Data

In [3]:
Top_50_Index = pd.read_csv('data/Top_50_Index.csv',index_col='Unnamed: 0')
Industy_Index = pd.read_csv('data/Industy_Index.csv',index_col='Unnamed: 0')
Overall_Index = pd.read_csv('data/Overall_Index.csv',index_col='Unnamed: 0')




"""
    CLOSE     قیمت پایانی
    FIRST      اولین قیمت
    HIGH     بیشترین قیمت
    LOW       کمترین قیمت
    LAST     آخرین معامله
    OPEN       قیمت دیروز
    OPENINT         تعداد
    VOL               حجم
    VALUE            ارزش
"""


df = pd.read_csv('data/Saderat.Bank.csv')



In [4]:
Overall_Index.head()

,Overall_Index
2010-03-27,12613.9
2010-03-28,12732.1
2010-03-29,12823.9
2010-03-30,12867.0
2010-03-31,12945.0


In [5]:
df.head()

,<TICKER>,<DTYYYYMMDD>,<FIRST>,<HIGH>,<LOW>,<CLOSE>,<VALUE>,<VOL>,<OPENINT>,<PER>,<OPEN>,<LAST>
0,Saderat.Bank,20200825,3390.0,3390.0,3390.0,3390.0,561296104080,165574072,3690,D,3230.0,3390.0
1,Saderat.Bank,20200824,3420.0,3430.0,3210.0,3230.0,6358063433250,1968425645,45800,D,3370.0,3350.0
2,Saderat.Bank,20200823,3580.0,3600.0,3360.0,3370.0,3197077552180,947959811,23334,D,3530.0,3360.0
3,Saderat.Bank,20200822,3740.0,3820.0,3460.0,3530.0,3289803479780,931416270,40409,D,3640.0,3480.0
4,Saderat.Bank,20200819,3700.0,3750.0,3630.0,3640.0,1303903815930,358278769,17330,D,3820.0,3630.0


In [6]:
df.columns = ['TICKER', 'DATE', 'FIRST', 'HIGH', 'LOW', 'CLOSE',
       'VALUE', 'VOL', 'OPENINT', 'PER', 'OPEN', 'LAST']


# Convert index(date) to jalali 

In [7]:
Overall_Index['Date'] = Overall_Index.index
Top_50_Index['Date'] = Top_50_Index.index
Industy_Index['Date'] = Industy_Index.index


Overall_Index['Date'] = Overall_Index['Date'].apply(lambda x: convert_date(x))
Top_50_Index['Date'] = Top_50_Index['Date'].apply(lambda x: convert_date(x))
Industy_Index['Date'] = Industy_Index['Date'].apply(lambda x: convert_date(x))


Overall_Index.index = Overall_Index['Date']
Top_50_Index.index = Top_50_Index['Date']
Industy_Index.index = Industy_Index['Date']

Overall_Index.drop('Date',axis = 1,inplace=True)
Top_50_Index.drop('Date',axis=1, inplace=True)
Industy_Index.drop('Date',axis=1, inplace=True)

df['DATE'] = df['DATE'].apply(lambda x: date_preprocess(x))
df['DATE'] = df['DATE'].apply(lambda x: convert_date(x))
df.index = df.DATE
df.drop('DATE',axis=1,inplace=True)


In [8]:
df.sort_index(inplace=True)
df = df.loc['1389-01-01':'1397-12-29']
df

,TICKER,FIRST,HIGH,LOW,CLOSE,VALUE,VOL,OPENINT,PER,OPEN,LAST
DATE,,,,,,,,,,,
1389-01-07,Saderat.Bank,1100.0,1100.0,1072.0,1096.0,92036293,84150,9,D,1096.0,1082.0
1389-01-08,Saderat.Bank,1094.0,1134.0,1073.0,1097.0,516624901,467582,36,D,1096.0,1100.0
1389-01-09,Saderat.Bank,1105.0,1127.0,1100.0,1098.0,279931740,252614,28,D,1097.0,1100.0
1389-01-10,Saderat.Bank,1102.0,1126.0,1092.0,1099.0,720040066,651862,36,D,1098.0,1100.0
1389-01-11,Saderat.Bank,1120.0,1120.0,1090.0,1099.0,297271310,271389,16,D,1099.0,1095.0
...,...,...,...,...,...,...,...,...,...,...,...
1397-12-13,Saderat.Bank,1249.0,1259.0,1224.0,1236.0,72074053563,58320975,3004,D,1232.0,1225.0
1397-12-14,Saderat.Bank,1236.0,1236.0,1185.0,1209.0,55705654482,46078691,2373,D,1236.0,1185.0
1397-12-15,Saderat.Bank,1213.0,1229.0,1170.0,1194.0,77233097825,64657910,3228,D,1209.0,1200.0


In [9]:
Overall_Index.head()

,Overall_Index
Date,
1389-01-07,12613.9
1389-01-08,12732.1
1389-01-09,12823.9
1389-01-10,12867.0
1389-01-11,12945.0


# Plot Index Data

In [43]:
import plotly
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected = True)

import plotly.graph_objs as go
from plotly.graph_objs import Layout
%matplotlib inline 

from IPython.display import display

In [45]:
data = go.Scatter(x=Overall_Index.index, y=Overall_Index.Overall_Index,mode='markers')
layout = go.Layout(calendar='jalali')
fig = go.Figure(data=data, layout=layout)
#plotly.offline.iplot(fig)
display(fig)

In [ ]:
fig = px.scatter(Top_50_Index,x=Top_50_Index.index, y=Top_50_Index.Top_50_Index,title='Top_50_Index')
fig.update_layout(calendar='jalali')
fig.show()

In [ ]:
fig = px.scatter(Industy_Index,x=Industy_Index.index, y=Industy_Index.Industy_Index,title='Industy_Index')
fig.update_layout(calendar='jalali')
fig.show()

In [ ]:
fig = px.scatter(df,x=df.index, y=df.CLOSE, title='Stock Close Price')
fig.update_layout(calendar='jalali')
fig.show()

# Merge dataframes in one

In [ ]:
df = pd.merge(df,Overall_Index,right_index=True,left_index=True)
df = pd.merge(df,Top_50_Index,right_index=True,left_index=True)
df = pd.merge(df,Industy_Index,right_index=True,left_index=True)
df.drop(['TICKER','PER'],axis=1,inplace=True)

# Compute RSI 14D Indicatores and Store in DataFrame

In [ ]:
df['RSI_14D'] = ta.momentum.rsi(close=df['LAST'],n=14,fillna=True)

In [ ]:
df

# Save Final DataFrame to CSV file

In [ ]:
df.to_csv('data/final_data.csv')